# Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
import sys

Mounted at /content/drive


## YOLOv7 Git 


In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1139, done.
remote: Total 1139 (delta 0), reused 0 (delta 0), pack-reused 1139
Receiving objects: 100% (1139/1139), 70.41 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (488/488), done.


In [2]:
cd /content/drive/MyDrive/braille2023/yolov7


/content/drive/MyDrive/braille2023/yolov7


# Train Code (전이학습)
--weight : 전이학습에 사용할 weight 파일경로  
--data : train set, val set의 경로와 class 설정 파일 경로  
--hyp : hyper-param과 augmentation에 사용할 param 파일 경로  
여기에 이미지 뒤집기 값은 0으로 바꿔줘야 함(점자가 겹침)  
--cfg는 모델의 구조. 사용하는 모델에 따라 다르게 설정.  
--name : train 과정에서 생성되는 파일들의 폴더 이름  


In [ ]:
# 1280을 입력으로 받는 모델을 학습시키려면 train_aux.py로 해야함, batch 2 learningrate 0.00125, 0.2, freeze backbone
!python train_aux.py --workers 8 --device 0 --batch-size 2 --epochs 100 --img 1280 1280 --data custom_data.yaml --hyp data/hyp.scratch.custom.yaml --cfg yolov7-w6-custom.yaml --name yolov7-w6-custom --weights yolov7-w6_training.pt --freeze 47

2023-06-09 06:47:44.821490: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 06:47:45.747345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
YOLOR 🚀 v0.1-122-g3b41c2c torch 2.0.1+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(weights='yolov7-w6_training.pt', cfg='yolov7-w6-custom.yaml', data='custom_data.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=50, batch_size=2, img_size=[1280, 1280], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='yolov7-w6-custo

# Detection Code

--weight를 테스트하고 싶은 모델 파라미터 파일로 바꿔야 함

In [6]:
!python detect.py --weights /content/drive/MyDrive/2023_1_Braille_Recognition/result/yolov7-w6-custom/weights/best.pt --conf 0.1 --img-size 1280 --source detect

Namespace(weights=['/content/drive/MyDrive/2023_1_Braille_Recognition/result/yolov7-w6-custom/weights/best.pt'], source='detect', img_size=1280, conf_thres=0.1, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-122-g3b41c2c torch 2.0.1+cu118 CPU

Fusing layers... 
IAuxDetect.fuse
Model Summary: 370 layers, 81376868 parameters, 0 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
3 as, 1 c, 3 is, 1 l, 1 n, 1 o, 2 ps, 1 s, 1 t, 2 uppers, Done. (3682.7ms) Infe

### Export

안드로이드 스튜디오 애플리케이션 상에서 모델을 사용할 수 있도록 best.pt -> best.ptl 파일로 변경(export)

In [7]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 81.1 MB/s eta 0:00:00


In [8]:
!python export.py --weights /content/drive/MyDrive/2023_1_Braille_Recognition/result/yolov7-w6-custom/weights/best.pt --grid --end2end --simplify --topk-all 100 --iou-thres 0.65 --conf-thres 0.35 --img-size 1280 1280 --max-wh 1280

Import onnx_graphsurgeon failure: No module named 'onnx_graphsurgeon'
Namespace(weights='/content/drive/MyDrive/2023_1_Braille_Recognition/result/yolov7-w6-custom/weights/best.pt', img_size=[1280, 1280], batch_size=1, dynamic=False, dynamic_batch=False, grid=True, end2end=True, max_wh=1280, topk_all=100, iou_thres=0.65, conf_thres=0.35, device='cpu', simplify=True, include_nms=False, fp16=False, int8=False)
YOLOR 🚀 v0.1-122-g3b41c2c torch 2.0.1+cu118 CPU

Fusing layers... 
IAuxDetect.fuse
Model Summary: 370 layers, 81376868 parameters, 0 gradients
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Starting TorchScript export with torch 2.0.1+cu118...
/content/drive/MyDrive/braille2023/yolov7/models/yolo.py:374: TracerWarning: Co